# Suppressing Warnings:
import warnings
warnings.filterwarnings("ignore")
import math

In [ ]:
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv2D, Dropout, Conv2DTranspose, concatenate
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D
import cv2
%matplotlib inline
import imutils
import os

In [ ]:
cd ..

In [ ]:
def dilateImage(image) :
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 255, 1)
    kernel = np.ones((3,3), np.uint8)
    erosion = cv2.erode(img, kernel, iterations=1)
    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(15)
    print("Original Image")
    plt.imshow(image, 'gray')
    plt.show()
    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(15)
    print("Eroded Image")
    plt.imshow(erosion, 'gray')
    plt.show()
    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(15)
    kernel = np.ones((5,5), np.uint8)
    dilation = cv2.dilate(erosion, kernel, iterations=1)
    print("Dilated Image")
    plt.imshow(dilation, 'gray')
    plt.show()
    return dilation
        

In [ ]:
img = cv2.imread('Examples/1891/a/a_1891_6.tif')
print("Original Image")
print(img.shape)
f = plt.figure()
f.set_figwidth(25)
f.set_figheight(15)
plt.imshow(img, 'gray')
plt.show()
img_binary = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY)
inverted_image = np.invert(img_binary[1])
print("Inverted Image")
f = plt.figure()
f.set_figwidth(25)
f.set_figheight(15)
plt.imshow(inverted_image, 'gray')
plt.show()
img1 = dilateImage(inverted_image)

f = plt.figure()
f.set_figwidth(25)
f.set_figheight(15)
#plt.imshow(img1, 'gray')

In [ ]:
#originalImage3 = cv2.imread('C:\\Users\\bharg\\Examples\\1871\\a\\a_1871_1.tif', cv2.IMREAD_GRAYSCALE)
#print(originalImage3)
%pwd
os.chdir('C:\\Users\\bharg')

In [ ]:
years = [1881, 1891, 1901, 1911, 1921]
folders = ['a','b','c']

for year in years :
    for folder in folders:
        os.chdir('C:\\Users\\bharg')
        DIR = 'Examples/'+str(year)+'/'+folder
        count = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
        for i in range(1, count) :
            #originalImage = []
            os.chdir('C:\\Users\\bharg')
            dir_file = 'Examples/'+str(year)+'/'+folder+'/'+folder+'_'+str(year)+'_'+str(i)+'.tif'
            print(dir_file)
            originalImage = cv2.imread('Examples/'+str(year)+'/'+folder+'/'+folder+'_'+str(year)+'_'+str(i)+'.tif', cv2.IMREAD_GRAYSCALE)
            print(originalImage)
            #originalImage1 = cv2.imread('Examples//1881//a//a_1881_'+str(i)+'.tif', cv2.COLOR_BGR2GRAY)
            #originalImage1 = dilateImage(originalImage1)
            #img_binary = cv2.threshold(originalImage1, 180, 255, cv2.THRESH_BINARY)
            #if originalImage1.shape[0] != 660:
            #print(originalImage1)
            #print(type(originalImage1))
            originalImage1 = cv2.resize(originalImage, (3600, 558))
            output_images = segmentImages(originalImage1, 1881)   
            saveImages(output_images, year, folder, i)
        


In [ ]:
def saveImages(output_images, year, folder, row):
    #directory = r'/Users/bharg/images/lineSegmentation_output/'+str(year)+'/'+folder
    directory = r'/Users/bharg/images/line_segmentation_all/output_2'
    count = 0;
    for i in output_images:
        os.chdir(directory) 
        #print("Before saving")   
        #print(os.listdir(directory))   
        #cv2.imwrite(str(year)+'_'+'row_'+str(row)+'_'+str(count)+'.tif', i)
        #shape = (1080, 1920)
        new_shape = (640,640)
        padded_image = np.pad(i, np.subtract(new_shape, i.shape), 'constant', constant_values=0) 
        cv2.imwrite(str(year)+'_'+folder+'_'+'row_'+str(row)+'_'+str(count)+'.png', padded_image)
        #print("After saving")  
        #print(os.listdir(directory))
        count += 1 
    

In [ ]:
#originalImage = cv2.imread('Examples//1891//a//_1.tif', cv2.IMREAD_GRAYSCALE)
##cv2.imshow('test_image', img)
#f = plt.figure()
#f.set_figwidth(25)
#f.set_figheight(15)
#plt.imshow(originalImage, cmap='Greys_r')
#plt.show()

In [ ]:
def getTemplateDetails(year, value) :
    
    coordinates = {}
    coordinates[1881] = [35, 192, 576, 952, 1132, 1421, 1570, 1955, 2342, 2900, 3195, 3576]
    coordinates[1891] = [34, 184, 564, 939, 1109, 1397, 1543, 1922, 2300, 2842, 3128, 3515]
    coordinates[1901] = [63, 212, 598, 981, 1126, 1413, 1560, 1941, 2321, 2867, 3155, 3546]
    coordinates[1911] = [26, 183, 573, 963, 1113, 1409, 1561, 1955, 2349, 2888, 3184, 3581]
    coordinates[1921] = [39, 194, 586, 974, 1122, 1414, 1563, 1950, 2340, 2873, 3168, 3562]
    
    years = coordinates.keys()
    
    template_areas = {}
    
    for i in years :
        year_coordinates = coordinates[i]
        temp_area_list = []
        for j in range(0, len(year_coordinates)-1) :
            temp_area_list.append(year_coordinates[j+1] - year_coordinates[j])
        template_areas[i] = temp_area_list
    if value == "area" :
        return template_areas[year]
    else :
        return coordinates[year]



In [ ]:
def segmentImages(image, year) :
    #(thresh, blackAndWhiteImage) = cv2.threshold(originalImage, 127, 255, cv2.THRESH_BINARY)
    thresh = 128
    
    # threshold the image
    originalImage = image
    img_binary = cv2.threshold(originalImage, 180, 255, cv2.THRESH_BINARY)
    inverted_image = np.invert(img_binary[1])
    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(15)
    #plt.imshow(img_binary[1], cmap='Greys_r')
    plt.imshow(inverted_image, cmap='Greys_r')
    plt.show()
    rows = inverted_image.shape[0]
    cols = inverted_image.shape[1]
    hist_horizontal_threshold = 24000
    horizontalSums = [0] * cols
    coordinates = []
    for c in range(0,cols):
        horizontalSums[c] = 0
        for r in range(0, rows) :
            horizontalSums[c] = horizontalSums[c] + inverted_image[r][c]            
    horizontalSums[0:13] = [1] *8
    coordinates = get_min_vertical_freq_indexes(horizontalSums, year)
    plt.plot(horizontalSums)
    plt.show()
    print(coordinates)
    hor_images = []
    f, axarr = plt.subplots(1,len(coordinates)-1)
    f.set_figwidth(25)
    f.set_figheight(15)
    for i in range(1, len(coordinates)) :
        axarr[i-1].imshow(inverted_image[:,coordinates[i-1]:coordinates[i]-5], cmap='Greys_r')
        hor_images.append(inverted_image[:,coordinates[i-1]:coordinates[i]-5])
        
    #print(coordinates)
    
    #print(len(hor_images))
    
    #Vertical Segmentations
    seg_images = []
    min_indexes = []
    
    for cnt in range(0, len(hor_images)) :
        rows = hor_images[cnt].shape[0]
        columns = hor_images[cnt].shape[1]
        
        rows_freq = []
        rows_freq_coord = []
        for row in range(0, rows -1):
            sum = 0
            for col in range(0, columns-1):
                sum = sum + hor_images[cnt][row][col]
            rows_freq.append(sum)
        min_indexes.append(get_min_freq_indexes(rows_freq))
        #plt.plot(rows_freq)
        #plt.show()
    
    #print(min_indexes)
    
    f, axarr = plt.subplots(1,len(min_indexes))
    f.set_figwidth(25)
    f.set_figheight(15)
    output_images = []
    for i in range(0, len(min_indexes)) :
        current_vert_img_coord = min_indexes[i]
        current_vert_img_coord.sort()
        axarr[i].imshow(hor_images[i][current_vert_img_coord[0]: current_vert_img_coord[1] , :], cmap='Greys_r')
        output_images.append(hor_images[i][current_vert_img_coord[0]: current_vert_img_coord[1] , :])
    plt.show()
    print("FINAL OUTPUT IMAGES ARRAY")
    return output_images
        

In [ ]:
def verityVericalSegmentation(col_nums_list, year) :
    vertital_gaps = []
    new_coordinates = []
    changed = False;
    template_area = getTemplateDetails(year, "area")
    template_coordinates = getTemplateDetails(year, "coordinates")
    
    if abs(col_nums_list[0] - template_coordinates[0]) > 20 :
        new_coordinates.append(template_coordinates[0])
        changed = True
    else :
        new_coordinates.append(col_nums_list[0])
        
    for i in range(0, len(col_nums_list)-1) :
        vertical_gap = col_nums_list[i+1] - col_nums_list[i]
        if (abs(vertical_gap - template_area[i]) > 30 or  changed) : 
            new_coordinates.append(new_coordinates[i]+template_area[i])
            changed = True
        else :
            new_coordinates.append(col_nums_list[i+1])
        vertital_gaps.append(new_coordinates[i+1] - new_coordinates[i])
    return new_coordinates
            

In [ ]:
def getAdjacentMaxFrequencyCoordinateValues(frequencyValueslist) :
    list_max = frequencyValueslist
    reversed = list_max[::-1]
    #print(reversed.index((max(reversed))))
    
    #print(list_max)
    start_index = list_max.index((max(list_max)))
    #print(start_index) 
    
    end_index = len(list_max)-reversed.index((max(reversed)))
    #print(end_index)
    
    
            
    #print(list_max[start_index : end_index])
    
    list_max[start_index : end_index] = [1]*len(list_max[start_index : end_index])
    
    #print(list_max)
    
    
    
    for i in range(0, 15) :
        #print(i)
        if start_index - (i+1) >= 0 : 
            list_max[start_index - (i+1)] = 1
        if end_index + (i+1) < len(list_max) :
            list_max[end_index-1 + (i+1)] = 1
            
    return  list_max
        

In [ ]:
def get_min_vertical_freq_indexes(cols_freq, year) :
    col_nums_list = []
    new_col_freq = cols_freq.copy()
    for i in range(0, 12) :
        min_value2 = max(new_col_freq)
        min_value_index_2 =  new_col_freq.index(min_value2)
        new_min_value_index_2 = new_col_freq.index(min_value2)
        leftIndexes = 15
        rightIndexes = 35
        
        value1 = 0
        value2 = 0
        
        if new_min_value_index_2 > leftIndexes:
            value1 = leftIndexes
        else :
            value1 = 0
        if new_min_value_index_2 + rightIndexes < len(new_col_freq):
            value2 = rightIndexes
        else :
            value2 = len(new_col_freq) - 1
            
        
        
        new_col_freq[new_min_value_index_2-value1 : new_min_value_index_2 + value2] = getAdjacentMaxFrequencyCoordinateValues(new_col_freq[new_min_value_index_2-value1 : new_min_value_index_2 + value2])
        #print(new_col_freq[new_min_value_index_2-6: new_min_value_index_2 + 6])
        #print('---')
        #print(min_value_index_2)
        col_nums_list.append(min_value_index_2)
    #min_value2 = max(new_rows_freq)
    #min_value_index_2 =  rows_freq.index(min_value2)
    #row_nums_list.append(min_value_index_1)
    #row_nums_list.append(min_value_index_2)
    #print(len(cols_freq))
    #print(len(new_col_freq))
    col_nums_list.sort()
    #print('======================')
    new_cols_list = verityVericalSegmentation(col_nums_list, year)
    return new_cols_list

In [ ]:
def get_min_freq_indexes(rows_freq) :
    row_nums_list = []
    min_value_index_1 = rows_freq.index(max(rows_freq))
    new_rows_freq = rows_freq
    
    leftIndexes = 15
    rightIndexes = 35
    
    value1 = 0
    value2 = 0
    
    if min_value_index_1 > leftIndexes:
        value1 = leftIndexes
    else :
        value1 = 0
    if min_value_index_1 + rightIndexes < len(new_rows_freq):
        value2 = rightIndexes
    else :
        value2 = len(new_rows_freq) - 1
    
    new_rows_freq[min_value_index_1-value1 : min_value_index_1 + value2]  = 
                    getAdjacentMaxFrequencyCoordinateValues(new_rows_freq[min_value_index_1-value1: min_value_index_1 + value2])
    min_value2 = max(new_rows_freq)
    min_value_index_2 =  new_rows_freq.index(min_value2)
    row_nums_list.append(min_value_index_1)
    row_nums_list.append(min_value_index_2)
    return row_nums_list